In [71]:
%load_ext autoreload
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [118]:
%autoreload 2

import sys,os,glob
import psutil

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
from pprint import pprint
from skimage import measure
from scipy.optimize import curve_fit

from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
from matplotlib.patches import Rectangle
import cartopy.crs as ccrs
from datetime import datetime as dt

import datetime as dt
import re
import gc
import matplotlib.animation as animation
import warnings

## current script object
thismodule = sys.modules[__name__]

# Own settings
from settings import *

workdir = os.getcwd()
moduledir, fcndir = defineDir(workdir)

Own modules available: ['casestudy', 'conditionalstats', 'conditionalstats_chunked', 'plot1D', 'plot2D', 'plotCondPDFs', 'lifecycle_mapping']
Own modules available: ['load_TOOCAN_DYAMOND_modif', 'load_TOOCAN_DYAMOND', 'load_TOOCAN_DYAMOND_modif_BF', 'fcns_load_RCEMIP_SAM', 'fcns_multiscale_extremes', 'PrecipGrid', 'fcns_load_DYAMOND_SAM', 'fcns_movie']


In [92]:
%autoreload 2

# to access segmentation files and simulation outputs
from fcns_load_DYAMOND_SAM import *
# to access TOOCAN objects
from load_TOOCAN_DYAMOND_modif_BF import *
# mapping function
from lifecycle_mapping import *

In [67]:
# load relation table
relation_table = loadRelTable('DYAMOND_SEG')

In [68]:
glob.glob(os.path.join(DIR_TOOCAN_DYAMOND,'*.gz'))

['/data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOBAL/2016/FileTracking/TOOCAN-GLOBAL-20160901-20160910.dat.gz',
 '/data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOBAL/2016/FileTracking/TOOCAN-GLOBAL-20160801-20160831.dat.gz']

In [69]:
toocan = loadAllMCSs(DIR_TOOCAN_DYAMOND,load_TOOCAN_DYAMOND)

load /data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOBAL/2016/FileTracking/TOOCAN-GLOBAL-20160901-20160910.dat.gz
/data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOBAL/2016/FileTracking/TOOCAN-GLOBAL-20160901-20160910.dat.gz
load /data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOBAL/2016/FileTracking/TOOCAN-GLOBAL-20160801-20160831.dat.gz
/data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOBAL/2016/FileTracking/TOOCAN-GLOBAL-20160801-20160831.dat.gz


In [70]:
# list of TOOCAN labels, for quicker mapping on toocan list
labels_toocan = [toocan[i].label for i in range(len(toocan))]

# Test LifeCycleMapping methods

In [9]:
# Example data

i_t = 500 # index in relation table
date = relation_table.loc[i_t].str_code

# load segmentation mask for that date
segmask = loadTOOCANSeg(i_t,relation_table)


/home/bfildier/.local/lib/python3.8/site-packages/xarray/conventions.py:492: SerializationWarning: variable 'MCS_label' has multiple fill values {-999.0, -998.0}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


In [10]:
# list all labels in segmask
def getLabelsInSegmentationMask(segmask):
    
    seg_1D = segmask.values.flatten()
    seg_1D_nonans = seg_1D[~np.isnan(seg_1D)]

    return np.unique(np.array(seg_1D_nonans,dtype=int))
    
labels_in_segmask = getLabelsInSegmentationMask(segmask)
print(labels_in_segmask)

[119749 121064 126715 ... 134180 134208 134331]


In [18]:
lcm = LifeCycleMapping(None,relation_table,toocan)

## dev getMCSAge(self,i_t,MCS_label)

In [15]:
# example MCS label
label = 134180

# index in list
i_MCS = np.where(np.array(labels_toocan) == label)[0][0]

print(i_MCS,lcm.toocan_index_of_label[label])

197682 126789


In [13]:
# show lifecycle

area_labels = 'surfkm2_172Wm2', 'surfkm2_132Wm2', 'surfkm2_110Wm2', 'surfkm2_90Wm2'
n_labs = len(area_labels)

fig,ax = plt.subplots(figsize=(6,5))

for i_l in range(n_labs):

    area_lab = area_labels[0]
    
    time = toocan[i_MCS].clusters.Localtime
    area_t = np.array(getattr(toocan[i_MCS].clusters,area_lab))
    
    ax.plot(time,area_t,label=area_lab)
    
plt.legend()

In [14]:
print(toocan[i_MCS].label)
bool(np.any(segmask == toocan[i_MCS].label).data)

134180


True

In [15]:
print(relation_table.loc[i_t].UTC)
print(relation_table.loc[i_t].str_code)
print(toocan[i_MCS].clusters.Utime == relation_table.loc[i_t].UTC)

17024.23
2016-8-11-11-30
[False  True False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False]


In [16]:
i_test = 500
time_i = toocan[i_test].Utime_Init
time_e = toocan[i_test].Utime_End
print('start_match =',toocan[i_test].clusters.Utime[0] == time_i)
print('end_match =',toocan[i_test].clusters.Utime[-1] == time_e)

start_match = True
end_match = True


In [17]:

def timeIndex2Timedelta(i_MCS,j_t_MCS):
    
    date_str = toocan[i_MCS].clusters.Utime[j_t_MCS]
    date_day = int(str(date_str).split('.')[0])
    date_30mn = int((str(date_str).split('.')[-1]).ljust(2,'0'))
    # compute time delta
    td = timedelta(days = int(date_str),seconds = date_30mn*30*60)
    # return
    return td


def getAgeMCS(i_MCS,j_t_MCS):
    """Arguments:
    - i_MCS: index of MCS in toocan list
    - j_t_MCS: time index in MCS lifecycle
    
    Returns:
    - age in hours"""
    
    # birth time
    time_birth = timeIndex2Timedelta(i_MCS,0)
    # current time
    time_now = timeIndex2Timedelta(i_MCS,j_t_MCS)
    # age delta
    age_delta = time_now - time_birth
    
    return age_delta.total_seconds()/3600


In [18]:
# find correct slice in TOOCAN lifecycle for current time
j_MCS_at_t = np.where(toocan[i_MCS].clusters.Utime == relation_table.loc[i_t].UTC)[0][0]

# turn it into system's age
age = getAgeMCS(i_MCS,j_MCS_at_t) # in hours

# total lifetime
lifetime_MCS = getAgeMCS(i_MCS,-1) # in hours

age/lifetime_MCS

0.02857142857142857

In [19]:
# check if current label occurs in current segmentation mask
time_match = bool(np.any(segmask == toocan[i_MCS].label).data)

if time_match:
    
    # find correct slice in TOOCAN lifecycle for current time
    j_MCS_at_t = np.where(toocan[i_MCS].clusters.Utime == relation_table.loc[i_t].UTC)[0][0]
    
    # turn it into system's age
    age_MCS_at_t = getAgeMCS(i_MCS,j_MCS_at_t) # in hours
    
    # total lifetime
    lifetime_MCS = getAgeMCS(i_MCS,-1) # in hours
    
    # normalized age
    norm_age_MCS_at_t = age_MCS_at_t/lifetime_MCS
    
    print('age:',age_MCS_at_t)
    print('duration:', lifetime_MCS)
    print('normalized age: %2.3f%%'%(norm_age_MCS_at_t*100))


age: 0.5
duration: 17.5
normalized age: 2.857%


## test getMCSAge(self,i_t,MCS_label)

In [109]:
lcm = LifeCycleMapping(None,relation_table,toocan)

In [110]:
# example time index
i_t_test = 500

# example MCS label
label_test = 134180

# load segmentation mask for that date
segmask = loadTOOCANSeg(i_t_test,relation_table)

/home/bfildier/.local/lib/python3.8/site-packages/xarray/conventions.py:492: SerializationWarning: variable 'MCS_label' has multiple fill values {-999.0, -998.0}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


In [112]:
age = lcm.computeMCSAgeMetrics(i_t_test,label_test,segmask)
print('age of MCS %d at time index %d:'%(label_test,i_t_test),age)

norm_age = lcm.computeMCSAgeMetrics(i_t_test,label_test,segmask,normalize=True)
print('Normalized age of MCS %d at time index %d:'%(label_test,i_t_test),norm_age)

age of MCS 134180 at time index 500: 0.5
Normalized age of MCS 134180 at time index 500: 0.02857142857142857


# test segMaskToAgeMask

In [122]:
from fcns_movie import *

In [123]:
lcm = LifeCycleMapping(None,relation_table,toocan)

In [116]:
# example time index
i_t_test = 500

# load segmentation mask for that date
segmask = loadTOOCANSeg(i_t_test,relation_table)

/home/bfildier/.local/lib/python3.8/site-packages/xarray/conventions.py:492: SerializationWarning: variable 'MCS_label' has multiple fill values {-999.0, -998.0}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


In [124]:
age_mask = lcm.computeAgeMask(i_t_test,segmask)

# show
fig = plt.figure(figsize=(Lx_fig,Ly_fig))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=0))

ax.pcolormesh(age_mask%10,transform=ccrs.PlateCarree(),cmap=cmap_mcs,alpha=1)


Unexpected exception formatting exception. Falling back to standard exception


/home/bfildier/analyses/explore-multiscale-extremes/modules/lifecycle_mapping.py:230: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
Traceback (most recent call last):
  File "/home/bfildier/.conda/envs/pyLMD/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_3803426/535694889.py", line 1, in <module>
    age_mask = lcm.computeAgeMask(i_t_test,segmask)
  File "/home/bfildier/analyses/explore-multiscale-extremes/modules/lifecycle_mapping.py", line 230, in computeAgeMask
    age_mask[segmask == label] = ages[label]
IndexError: boolean index did not match indexed array along dimension 0; dimension is 1 but corresponding boolean dimension is 2072

During hand